In [1]:
%env KERAS_BACKEND=tensorflow
%env CUDA_HOME=/usr/local/cuda-7.5

env: KERAS_BACKEND=tensorflow
env: CUDA_HOME=/usr/local/cuda-7.5


In [2]:
import os
import csv
import re
import pickle
import random
import math
import dicom
import numpy as np
from tqdm import tqdm
from natsort import natsorted
from skimage import transform
from sklearn.externals import joblib
from scipy import ndimage
from matplotlib import path

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import pickle
import numpy as np
import h5py
from sklearn.externals import joblib
import random
from keras.models import Sequential, Graph
from keras.layers.core import Layer, Activation, Dense, Dropout, Flatten, Merge, Reshape, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ParametricSoftplus, ELU
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
import tensorflow as tf

# for preventing python max recursion limit error
import sys
sys.setrecursionlimit(50000)

Using TensorFlow backend.


In [10]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
x = tf.clip_by_value(tf.mul(tf.sub(np.array([[[-0.3,-0.1],[0.0,0.2]]], dtype=np.float32), -0.1), 1e6), 0.0, 1.0)
print(sess.run(x))

[[[ 0.  0.]
  [ 1.  1.]]]


In [4]:
(data_ED_train, data_ED_train_val, 
 data_ES_train, data_ES_train_val) = joblib.load('../../data_proc/trainset2_data_for_thresholding_volume_net.pkl')

In [5]:
thresh_optimizer_ED = Sequential()

thresh_optimizer_ED.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf', 
                                      input_shape=(96, 96, 1)))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(MaxPooling2D(pool_size=(2,2), border_mode='valid', dim_ordering='tf'))

thresh_optimizer_ED.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(Convolution2D(32, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(MaxPooling2D(pool_size=(2,2), border_mode='valid', dim_ordering='tf'))

thresh_optimizer_ED.add(Convolution2D(64, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(Convolution2D(64, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(Convolution2D(64, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(MaxPooling2D(pool_size=(2,2), border_mode='valid', dim_ordering='tf'))

thresh_optimizer_ED.add(Convolution2D(128, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(Convolution2D(128, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(Convolution2D(128, 3, 3, init='he_uniform', border_mode='same', dim_ordering='tf'))
thresh_optimizer_ED.add(BatchNormalization())
thresh_optimizer_ED.add(ELU())
thresh_optimizer_ED.add(MaxPooling2D(pool_size=(2,2), border_mode='valid', dim_ordering='tf'))

thresh_optimizer_ED.add(Flatten())
thresh_optimizer_ED.add(Dense(1024, activation='relu'))
thresh_optimizer_ED.add(Dropout(0.5))
thresh_optimizer_ED.add(Dense(1, activation='linear'))

In [6]:
volumes_ED = Graph()

NUM_SLICES = 20

def calc_volume(input_dict):
    import tensorflow as tf
    segment, multiplier, threshold = input_dict.values()
    threshold_broadcasted = tf.expand_dims(tf.expand_dims(tf.clip_by_value(threshold, 0.00001, 0.5), 2), 3)
    return tf.mul(tf.reduce_sum(tf.clip_by_value(tf.mul(tf.sub(segment, threshold_broadcasted), 1e6), 0.0, 1.0),  
                                [1, 2], keep_dims=False), multiplier)

for i in range(NUM_SLICES):
    volumes_ED.add_input(name='input_{}'.format(i), input_shape=(96, 96, 1), dtype='float')
for i in range(NUM_SLICES):
    # per-slice multiplier: combo of z-depth, pixel spacing, scaling factor
    volumes_ED.add_input(name='multiplier_{}'.format(i), input_shape=(1,), dtype='float')  

volumes_ED.add_shared_node(thresh_optimizer_ED, 
                           name='thresholder', 
                           inputs=['input_{}'.format(i) for i in range(NUM_SLICES)],
                           outputs=['threshold_{}'.format(i) for i in range(NUM_SLICES)])

for i in range(NUM_SLICES):
    volumes_ED.add_node(Lambda(calc_volume, output_shape=(1,)), 
                        name='slice_volume_{}'.format(i),
                        inputs=['input_{}'.format(i), 'multiplier_{}'.format(i), 'threshold_{}'.format(i)], 
                        merge_mode='join')

volumes_ED.add_output(name='volume', 
                      inputs=['slice_volume_{}'.format(i) for i in range(NUM_SLICES)], 
                      merge_mode='sum')

def MAE(y_true, y_pred):
    
volumes_ED.compile('adam', {'volume': MAE})

In [7]:
batch_size = 16
nb_epoch = 100

earlystopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
checkpointer = ModelCheckpoint(filepath='../../model_weights/weights_opt_thresh_volume_ED.hdf5',
                               verbose=1, save_best_only=True)

volumes_ED.fit(data_ED_train,
               batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, shuffle=True,
               validation_data=data_ED_train_val,
               #callbacks=[checkpointer, earlystopping])
               callbacks=[earlystopping])

Train on 450 samples, validate on 50 samples
Epoch 1/100
450/450 [==============================] - 130s - loss: 35.0806 - val_loss: 18.1241
Epoch 2/100
450/450 [==============================] - 127s - loss: 36.3120 - val_loss: 18.1452
Epoch 3/100
450/450 [==============================] - 127s - loss: 36.3122 - val_loss: 18.1441
Epoch 4/100
450/450 [==============================] - 127s - loss: 36.3125 - val_loss: 18.1442
Epoch 5/100
450/450 [==============================] - 128s - loss: 36.3124 - val_loss: 18.1440
Epoch 6/100
144/450 [========>.....................] - ETA: 85s - loss: 37.1344

KeyboardInterrupt: 

In [15]:
j = 100
test_obj = {}
for i in range(NUM_SLICES):
    test_obj['input_{}'.format(i)] = data_ED_train['input_{}'.format(i)][j:j+2]
    test_obj['multiplier_{}'.format(i)] = data_ED_train['multiplier_{}'.format(i)][j:j+2]
print(volumes_ED.predict(test_obj)['volume'])
print(data_ED_train['volume'][j:j+2])
print(np.mean(np.abs(volumes_ED.predict(test_obj)['volume'] - data_ED_train['volume'][j:j+2])))

18.1440235901


In [19]:
np.max(data_ED_train['input_0'])

3.758621